In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import serial
cv2.startWindowThread()
%matplotlib inline

ser = serial.Serial()
ser.baudrate = 115200
ser.port = 'COM7'
ser.open()
ser

Serial<id=0x2904be005c0, open=True>(port='COM7', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)

In [2]:
red_low_1 = np.array([0,150,95])
red_high_1 = np.array([10,255,255])
red_low_2 = np.array([0,150,95])
red_high_2 = np.array([160,255,255])
green_low = np.array([40,20,190])
green_high = np.array([70,255,255])
orange_low = np.array([10,50,190])
orange_high = np.array([40,255,255])
white_low = np.array([80,0,230])
white_high = np.array([110,20,255])
blue_low = np.array([70,100,220])
blue_high = np.array([130,255,255])

In [3]:
# CAMERA
cap = cv2.VideoCapture(0)

while(1):
    
    # Capture the scene; detect blue, white and green; detect contours; compute centers;
    
    _, frame = cap.read()
    imhsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    imc = frame.copy()

    # blue contours and centroids
    mask1 = cv2.inRange(imhsv, blue_low, blue_high)
    
    image, contours, hierarchy = cv2.findContours(mask1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        imc = cv2.drawContours(imc, [cnt], 0, (255,0,0), 3)
    moments  = [cv2.moments(cnt) for cnt in contours]
    centroids_blue = []
    for m in moments:
        try:
            centroids_blue += [( int(round(m['m10']/m['m00'])),int(round(m['m01']/m['m00'])) )]
        except:
            pass
    for c in centroids_blue:
        cv2.circle(imc, c, 3, (200,0,0))
        
    # white contours and centroids
    mask2 = cv2.inRange(imhsv, white_low, white_high)

    image, contours, hierarchy = cv2.findContours(mask2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        imc = cv2.drawContours(imc, [cnt], 0, (0,0,255), 3)
    moments  = [cv2.moments(cnt) for cnt in contours]
    centroids_white = []
    for m in moments:
        try:
            centroids_white += [( int(round(m['m10']/m['m00'])),int(round(m['m01']/m['m00'])) )]
        except:
            pass
    for c in centroids_white:
        cv2.circle(imc, c, 3, (0,0,200))
    
    # green (robot) contours and centroids
    mask3 = cv2.inRange(imhsv, green_low, green_high)

    image, contours, hierarchy = cv2.findContours(mask3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        imc = cv2.drawContours(imc, [cnt], 0, (0,255,0), 3)
        
    contours_green = sorted(contours, key = cv2.contourArea, reverse = True)[:3]
    main_cnt = contours_green[0]
    
    try:
        center, axis, angle = cv2.fitEllipse(main_cnt)

        hyp = 100  # length of the orientation line

        # Find out coordinates of 2nd point if given length of line and center coord 
        linex = int(center[0]) + int(np.sin(np.radians(angle))*hyp)
        liney = int(center[1]) - int(np.cos(np.radians(angle))*hyp)

        # Draw orienation
        cv2.line(imc, (int(center[0]),int(center[1])), (linex, liney), (0,0,255),5)             
        cv2.circle(imc, (int(center[0]), int(center[1])), 10, (255,0,0), -1)
    except:
        pass
        
    moments  = [cv2.moments(cnt) for cnt in contours_green]
    centroids_green = []
    for m in moments:
        try:
            centroids_green += [( int(round(m['m10']/m['m00'])),int(round(m['m01']/m['m00'])) )]
        except:
            pass
    for c in centroids_green:
        cv2.circle(imc, c, 3, (0,200,0))
    
    print(centroids_green)
    
    # Output Commads via serial port to the Controller
    
    # ser.write(b's')
    
    
    
    # show the final frame    
    
    cv2.imshow('c', imc)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    
cv2.destroyAllWindows()

[(362, 216), (473, 258), (479, 216)]
[(362, 216), (473, 258), (478, 216)]
[(362, 215), (473, 257), (478, 216)]
[(362, 215), (474, 257), (479, 215)]
[(362, 215), (475, 258), (480, 216)]
[(362, 215), (475, 258), (479, 216)]
[(362, 215), (479, 216), (475, 257)]
[(362, 215), (474, 258), (478, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (479, 216), (475, 257)]
[(362, 215), (475, 258), (479, 216)]
[(362, 215), (479, 216), (475, 258)]
[(362, 215), (478, 216), (474, 257)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (475, 257), (480, 216)]
[(362, 215), (475, 258), (480, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (478, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (475, 257), (479, 216)]
[(362, 215), (475, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[

[(362, 215), (474, 257), (479, 215)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (479, 216), (474, 258)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (478, 216), (474, 257)]
[(362, 215), (475, 258), (480, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (479, 216), (475, 258)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (478, 216), (474, 257)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (475, 258), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (475, 258), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[

[(362, 215), (479, 216), (475, 257)]
[(362, 215), (478, 216), (474, 257)]
[(361, 215), (478, 216), (474, 257)]
[(361, 215), (474, 257), (479, 216)]
[(362, 215), (479, 216), (475, 257)]
[(362, 215), (474, 257), (480, 215)]
[(362, 215), (475, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(361, 215), (474, 258), (478, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 257), (479, 215)]
[(362, 215), (474, 257), (480, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (479, 216), (474, 257)]
[(362, 215), (474, 257), (480, 216)]
[(362, 215), (475, 258), (480, 216)]
[(362, 215), (479, 216), (475, 258)]
[(362, 215), (474, 257), (478, 216)]
[(362, 215), (474, 257), (479, 215)]
[(362, 215), (479, 216), (474, 257)]
[(362, 215), (475, 257), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 257), (479, 215)]
[(362, 215), (474, 257), (479, 216)]
[

[(362, 215), (475, 258), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (479, 216), (475, 258)]
[(362, 215), (479, 216), (475, 258)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (478, 216), (475, 257)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (479, 216), (475, 257)]
[(361, 215), (478, 216), (474, 257)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (479, 216), (475, 258)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (475, 258), (479, 216)]
[(362, 215), (475, 258), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 257), (479, 215)]
[(362, 215), (474, 257), (478, 216)]
[(362, 215), (474, 257), (478, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[

[(362, 215), (475, 257), (479, 216)]
[(362, 215), (479, 216), (475, 257)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (478, 215), (475, 257)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 258), (480, 216)]
[(362, 215), (479, 216), (475, 257)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 257), (479, 215)]
[(362, 215), (473, 257), (479, 215)]
[(362, 215), (479, 216), (475, 257)]
[(362, 215), (475, 257), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 257), (479, 215)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (479, 215)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (479, 215), (475, 257)]
[(362, 215), (475, 257), (479, 215)]
[(362, 215), (474, 257), (479, 215)]
[(362, 215), (478, 215), (475, 257)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (479, 216), (474, 257)]
[(362, 215), (474, 257), (479, 216)]
[

...

[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 258), (478, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (475, 258), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 258), (478, 216)]
[(362, 215), (474, 257), (478, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (479, 216), (474, 258)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (479, 216), (474, 258)]
[(362, 215), (478, 216), (474, 258)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 257), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[

[(362, 215), (479, 216), (474, 258)]
[(362, 215), (479, 216), (475, 258)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (479, 216), (474, 258)]
[(362, 215), (474, 258), (478, 216)]
[(362, 215), (479, 216), (475, 258)]
[(362, 215), (479, 217), (474, 258)]
[(362, 215), (474, 258), (479, 217)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (474, 258), (479, 216)]
[(362, 215), (478, 216), (474, 258)]
[(362, 215), (479, 216), (475, 258)]
[(363, 215), (474, 258), (479, 216)]
[(362, 215), (481, 216), (474, 254)]
[(363, 215), (481, 216), (476, 257)]
[(362, 216), (482, 217), (476, 257)]
[(363, 216), (481, 215), (482, 256)]
[(362, 216), (481, 216), (480, 256)]
[(362, 216), (482, 216), (477, 256)]
[(362, 216), (485, 216), (474, 255)]
[(362, 216), (482, 216), (470, 254)]
[(362, 216), (484, 216), (481, 253)]
[(362, 216), (478, 216), (477, 254)]
[(363, 216), (479, 216), (487, 259)]
[(362, 216), (479, 217), (479, 258)]
[(363, 215), (472, 218), (469, 255)]
[(365, 215), (480, 217), (476, 254)]
[

[(366, 205), (482, 222), (472, 262)]
[(366, 206), (483, 222), (472, 262)]
[(366, 205), (471, 263), (483, 222)]
[(366, 205), (482, 222), (472, 262)]
[(365, 205), (471, 262), (482, 222)]
[(365, 205), (482, 222), (471, 262)]
[(366, 206), (471, 262), (482, 223)]
[(366, 205), (483, 222), (471, 262)]
[(366, 205), (483, 222), (471, 262)]
[(365, 206), (471, 262), (483, 222)]
[(365, 205), (482, 222), (471, 262)]
[(365, 206), (482, 222), (471, 262)]
[(366, 205), (483, 222), (472, 262)]
[(366, 206), (483, 222), (471, 262)]
[(366, 205), (483, 222), (471, 262)]
[(366, 206), (471, 262), (482, 223)]
[(365, 206), (482, 222), (471, 262)]
[(366, 206), (483, 222), (471, 262)]
[(366, 206), (482, 222), (471, 262)]
[(366, 205), (483, 222), (472, 262)]
[(366, 205), (471, 262), (483, 222)]
[(366, 206), (482, 222), (471, 262)]
[(365, 205), (483, 222), (470, 262)]
[(365, 205), (482, 222), (471, 262)]
[(366, 205), (483, 222), (472, 262)]
[(366, 206), (471, 262), (483, 222)]
[(366, 206), (483, 222), (472, 262)]
[

[(366, 205), (482, 222), (471, 262)]
[(366, 205), (483, 222), (472, 262)]
[(366, 205), (471, 262), (483, 222)]
[(366, 206), (482, 222), (472, 262)]
[(366, 205), (482, 222), (471, 262)]
[(365, 205), (471, 262), (483, 222)]
[(366, 206), (483, 222), (471, 262)]
[(366, 206), (483, 222), (471, 262)]
[(366, 206), (482, 222), (471, 262)]
[(366, 205), (482, 222), (472, 262)]
[(366, 205), (482, 222), (471, 262)]
[(366, 206), (482, 223), (471, 262)]
[(366, 205), (482, 222), (471, 262)]
[(366, 206), (483, 222), (471, 262)]
[(366, 206), (483, 222), (471, 262)]
[(366, 206), (482, 222), (471, 262)]
[(366, 205), (483, 222), (471, 262)]
[(366, 205), (481, 222), (471, 262)]
[(366, 206), (471, 262), (483, 222)]
[(366, 205), (482, 222), (471, 262)]
[(366, 205), (483, 222), (471, 262)]
[(366, 205), (482, 222), (471, 262)]
[(366, 205), (483, 222), (470, 262)]
[(366, 206), (483, 222), (471, 262)]
[(366, 205), (483, 223), (471, 262)]
[(366, 205), (482, 222), (470, 261)]
[(365, 205), (471, 262), (482, 222)]
[

KeyboardInterrupt: 

In [ ]:
ser.write(b'w')

In [ ]:
ser.write(b'z')

In [1]:
centers = [[(362, 215), (475, 257), (479, 216)]]